In [1]:
#!pip install bs4==0.0.1 #if running in virtual notebook 

In [23]:
#Import the libraries

from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta, date
import re

In [24]:
#Get the webpage and parse it as html

page = requests.get("https://weather.com/weather/tenday/l/San+Francisco+CA?canonicalCityId=dfdaba8cbe3a4d12a8796e1f7b1ccc7174b4b0a2d5ddb1c8566ae9f154fa638c")
soup = BeautifulSoup(page.content, 'html.parser')

In [46]:
#Retrieve the day names nested in body 

day_names = soup.find_all('h3', class_ = 'DailyContent--daypartName--3G5Y8') #day names are in this tag

list_of_days = []
for each_day in day_names:
    e = each_day.text                #'.text' returns the text inside the Tag
    list_of_days.append(e)

list_of_days = list_of_days[0:10]    #only getting the first 10 days details

#First pattern for the DATE
date_string = ' '.join(list_of_days)  
date = re.findall("[A-Z][a-z]+ [0-9]+", date_string) #use regex to filter the date 

#Second pattern for Day and Night. 
first_day = list_of_days[0]           #check for first day and create a list based on that
day_checker =re.findall(" [A-Z][a-z]+",first_day)
print(day_checker)

if ' Night' in day_checker:
    type_of_day = ['Night','Day']*5   
elif ' Day' in day_checker:
    type_of_day = ['Day','Night']*5   #Day and Night is a repetitive pattern, so creating it is easy rather than extracting it 
print(type_of_day)

[' Night']
['Night', 'Day', 'Night', 'Day', 'Night', 'Day', 'Night', 'Day', 'Night', 'Day']


In [5]:
#Retrieve the weather summary nested inside the body and find temperatures

day_temp = soup.find_all('div', class_ = 'DetailsSummary--temperature--3FMlw') 

temp_list = []
for summary in day_temp:
    e = summary.text              
    temp_list.append(e)

#Extract the temperature by searching for a regex pattern and filter it further
temp_string = ' '.join(temp_list)            
temp = re.findall("([0-9]\d[°])", temp_string)      
temp = ' '.join(temp).replace('°',' ').split()     
print(temp)


['45', '63', '42', '63', '44', '59', '47', '62', '48', '67', '51', '71', '50', '69', '47', '63', '46', '60', '45', '58', '45', '59', '44', '58', '44', '59', '44', '58', '44']


In [6]:
#Convert Fahrenheit to celcius 

temp_celcius = []
def celcius(temp):
    for each_temp in temp:
        each_temp = (int(each_temp)- 32) * (5/9)    #converting to int type as it gives an error for float
        new_temp = round(each_temp)
        temp_celcius.append(f'{new_temp} °C')
    return temp_celcius
celcius(temp)

temp_celcius = temp_celcius[0:10]

In [7]:
#Extracting the summary description
summary = soup.find_all('p', class_ = 'DailyContent--narrative--3AcXd')

summary_list = []
for summary in summary:
    e = summary.text              
    summary_list.append(e)

summary_list = summary_list[0:10]

In [50]:
#creating the dataf

df = pd.DataFrame({'Date':date,'Day':type_of_day,'Temp':temp_celcius,'summary':summary_list}) 
print(df)

#df.to_csv('Weather_forecast_Hard.csv', index=False) #write the df to a csv file

     Date    Day   Temp                                            summary
0  Thu 25  Night   7 °C  Clear skies. Low around 45F. Winds N at 5 to 1...
1  Fri 26    Day  17 °C  Sunny. High 63F. NNE winds shifting to WNW at ...
2  Fri 26  Night   6 °C           Clear. Low 42F. Winds SW at 5 to 10 mph.
3  Sat 27    Day  17 °C  Mainly sunny. High 63F. SSE winds shifting to ...
4  Sat 27  Night   7 °C  A mostly clear sky. Low 44F. Winds SW at 5 to ...
5  Sun 28    Day  15 °C  Sunshine along with some cloudy intervals. Hig...
6  Sun 28  Night   8 °C  A few clouds from time to time. Low 47F. Winds...
7  Mon 29    Day  17 °C  A mainly sunny sky. High 62F. Winds WNW at 10 ...
8  Mon 29  Night   9 °C  A mostly clear sky. Low 48F. Winds NNW at 10 t...
9  Tue 30    Day  19 °C  A mainly sunny sky. High 67F. Winds N at 10 to...
